In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xgboost as xgb
from os.path import join as opjoin
from resonancesml.commands.shortcuts import classify
import sklearn as skl

In [2]:
def _cutoff(catalog_data, by_column, interval=(0.25, 0.75), interval_ext=(1.5, 1.5)):
    col_name = by_column
    quantile1_len = int(catalog_data.shape[0] * interval[0])
    quantile3_len = int(catalog_data.shape[0] * interval[1])
    sorted_catalog_data = catalog_data[col_name].sort_values()
    s_quantile1 = sorted_catalog_data[:quantile1_len].tail(1).values[0]
    s_quantile3 = sorted_catalog_data[:quantile3_len].tail(1).values[0]

    inter_quantile = s_quantile3 - s_quantile1
    bottom_threshold = s_quantile1 - interval_ext[0] * inter_quantile
    top_threshold = s_quantile3 + interval_ext[1] * inter_quantile

    mask = (catalog_data[col_name] >= bottom_threshold) & (catalog_data[col_name] <= top_threshold)
    return mask

def cutoff_outliers(catalog_data, by_columns=[], interval=(0.25, 0.75), interval_ext=(1.5, 1.5)):
    mask = None
    for col in by_columns:
        col_mask = _cutoff(catalog_data, col, interval, interval_ext)
        if mask is None:
            mask = col_mask
        else:
            mask &= col_mask
    return catalog_data[mask]

In [3]:
FOLDER = '4J-2S-1-pure'
RESONANCE_AXIS = 2.39

In [3]:
FOLDER = '5J-2S-2-pure'
RESONANCE_AXIS = 3.1747

In [4]:
CATALOG_PATH = opjoin('input', 'catalogs', 'all.syn')
AXIS_SWING = 0.01
BOT_AXIS = RESONANCE_AXIS - AXIS_SWING
TOP_AXIS = RESONANCE_AXIS + AXIS_SWING
librator_list_path = opjoin('input', 'librations', FOLDER, 'first50')
DATASET_HEADER = [
    'mag.',
    'a',
    'e',
    'sin I',
    'n',
    'g ("/yr)',
    's ("/yr)',
    'LCEx1E6',
    'My',
]
dtype = {x: float for x in DATASET_HEADER}
catalog_data = pd.read_csv(
    CATALOG_PATH,
    index_col=0,
    delim_whitespace=True,
    skiprows=2,
    names=DATASET_HEADER,
    dtype=dtype,
    nrows=406251,
)
catalog_data = catalog_data[(catalog_data['a'] >= BOT_AXIS) & (catalog_data['a'] <= TOP_AXIS)]

librator_numbers = pd.read_csv(librator_list_path, names=None, header=None, dtype=int)
last_librator = librator_numbers[0].tail(1).values[0]
catalog_data = catalog_data[catalog_data.index <= last_librator]
catalog_data

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My
106,7.48,3.170823,0.144660,0.062489,63.733326,162.988895,-104.807367,22.63,-2.0
154,7.61,3.184401,0.131878,0.344440,63.336366,90.525026,-70.743921,2.56,-2.0
176,8.21,3.177621,0.192273,0.373321,63.540540,74.039576,-76.203024,34.33,-2.0
184,8.42,3.181990,0.103860,0.037295,63.400196,175.916558,-96.226761,1.48,-2.0
199,8.54,3.171808,0.200189,0.233687,63.712226,125.576699,-102.860299,32.83,10.0
297,8.99,3.170821,0.119254,0.141405,63.736709,143.662833,-92.391755,18.37,-2.0
316,9.95,3.174991,0.136681,0.023818,63.615776,170.153854,-104.059614,81.68,10.0
408,9.11,3.164864,0.118526,0.180358,63.918097,126.311625,-87.877084,1.16,-2.0
490,8.47,3.174039,0.065555,0.159259,63.641959,137.566983,-82.560291,98.28,10.0
511,6.30,3.174155,0.189688,0.246203,63.636785,122.290353,-97.760181,79.10,10.0


In [5]:
catalog_data = catalog_data.assign(
    qwe=catalog_data['LCEx1E6'] * catalog_data['n'] * catalog_data['e'],
    asd=catalog_data['s ("/yr)'] * catalog_data['g ("/yr)'],
    a0=catalog_data['a'] - RESONANCE_AXIS
)
# catalog_data = cutoff_outliers(catalog_data, ['s ("/yr)', 'e', 'LCEx1E6', 'qwe', 'asd'])

# catalog_data[DATASET_HEADER] = catalog_data[DATASET_HEADER] - catalog_data[DATASET_HEADER].min()
# catalog_data[DATASET_HEADER] = catalog_data[DATASET_HEADER] / catalog_data[DATASET_HEADER].max()

librators_mask = (catalog_data.index.isin(librator_numbers[0]))
catalog_data = catalog_data.assign(librator=librators_mask)

librators = catalog_data[librators_mask]
non_librators = catalog_data[~librators_mask]
catalog_data

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My,a0,asd,qwe,librator
106,7.48,3.170823,0.144660,0.062489,63.733326,162.988895,-104.807367,22.63,-2.0,-0.003877,-17082.436935,208.641117,False
154,7.61,3.184401,0.131878,0.344440,63.336366,90.525026,-70.743921,2.56,-2.0,0.009701,-6404.095288,21.382908,False
176,8.21,3.177621,0.192273,0.373321,63.540540,74.039576,-76.203024,34.33,-2.0,0.002921,-5642.039587,419.414954,False
184,8.42,3.181990,0.103860,0.037295,63.400196,175.916558,-96.226761,1.48,-2.0,0.007290,-16927.880583,9.745459,False
199,8.54,3.171808,0.200189,0.233687,63.712226,125.576699,-102.860299,32.83,10.0,-0.002892,-12916.856807,418.730220,False
297,8.99,3.170821,0.119254,0.141405,63.736709,143.662833,-92.391755,18.37,-2.0,-0.003879,-13273.261269,139.627635,False
316,9.95,3.174991,0.136681,0.023818,63.615776,170.153854,-104.059614,81.68,10.0,0.000291,-17706.144368,710.215223,False
408,9.11,3.164864,0.118526,0.180358,63.918097,126.311625,-87.877084,1.16,-2.0,-0.009836,-11099.897280,8.788080,False
490,8.47,3.174039,0.065555,0.159259,63.641959,137.566983,-82.560291,98.28,10.0,-0.000661,-11357.570148,410.027062,False
511,6.30,3.174155,0.189688,0.246203,63.636785,122.290353,-97.760181,79.10,10.0,-0.000545,-11955.127044,954.826737,False


In [6]:
# from sklearn.decomposition import PCA
dataset = catalog_data.ix[:, :-1]
dataset
# pca = None
# for i in range(0, dataset.shape[1]):
#     pca = PCA(i)
#     pca.fit(dataset)
#     print(i, pca.explained_variance_ratio_)
#     if sum(pca.explained_variance_ratio_) > 0.9:
#         print(i)
#         break
# transformed_features = pca.transform(dataset)
# np.corrcoef(transformed_features[:, 3], librators_mask)
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(ratio=0.99, random_state=42)
# features, targets = sm.fit_sample(dataset, librators_mask)
# names = catalog_data.columns[:-1]
# features = pd.DataFrame(features, columns=names)
# features[targets]

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My,a0,asd,qwe
106,7.48,3.170823,0.144660,0.062489,63.733326,162.988895,-104.807367,22.63,-2.0,-0.003877,-17082.436935,208.641117
154,7.61,3.184401,0.131878,0.344440,63.336366,90.525026,-70.743921,2.56,-2.0,0.009701,-6404.095288,21.382908
176,8.21,3.177621,0.192273,0.373321,63.540540,74.039576,-76.203024,34.33,-2.0,0.002921,-5642.039587,419.414954
184,8.42,3.181990,0.103860,0.037295,63.400196,175.916558,-96.226761,1.48,-2.0,0.007290,-16927.880583,9.745459
199,8.54,3.171808,0.200189,0.233687,63.712226,125.576699,-102.860299,32.83,10.0,-0.002892,-12916.856807,418.730220
297,8.99,3.170821,0.119254,0.141405,63.736709,143.662833,-92.391755,18.37,-2.0,-0.003879,-13273.261269,139.627635
316,9.95,3.174991,0.136681,0.023818,63.615776,170.153854,-104.059614,81.68,10.0,0.000291,-17706.144368,710.215223
408,9.11,3.164864,0.118526,0.180358,63.918097,126.311625,-87.877084,1.16,-2.0,-0.009836,-11099.897280,8.788080
490,8.47,3.174039,0.065555,0.159259,63.641959,137.566983,-82.560291,98.28,10.0,-0.000661,-11357.570148,410.027062
511,6.30,3.174155,0.189688,0.246203,63.636785,122.290353,-97.760181,79.10,10.0,-0.000545,-11955.127044,954.826737


In [ ]:
plt.figure(FOLDER, figsize=(15, 15))
x_col = 'a'
y_col = 'e'
# axes = plt.gca()
# axes.set_xlim([-0.003,0.003])
# axes.set_ylim([-0.5,5])
plt.scatter(
    non_librators[x_col],
    non_librators[y_col],
    marker='^', color='r')
# plt.scatter(
#     librators[x_col], 
#     librators[y_col],
#     marker='o', color='b')
plt.xlabel(x_col, fontsize=18)
plt.ylabel(y_col, fontsize=18)
plt.legend(['non librators', 'librators'], loc='upper left')
plt.title(FOLDER, fontsize=20)
plt.show()

In [11]:
catalog_data

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My,a0,asd,qwe,librator
106,0.122661,0.306315,0.485340,0.110348,0.681375,0.630235,0.599083,0.132030,0.0,-0.003877,-17082.436935,208.641117,False
154,0.136175,0.986210,0.441747,0.698191,0.031935,0.298798,0.857352,0.014936,0.0,0.009701,-6404.095288,21.382908,False
176,0.198545,0.646703,0.647732,0.758407,0.365971,0.223397,0.815961,0.200292,0.0,0.002921,-5642.039587,419.414954,False
184,0.220374,0.865477,0.346188,0.057821,0.136363,0.689364,0.664141,0.008635,0.0,0.007290,-16927.880583,9.745459,False
199,0.232848,0.355665,0.674730,0.467283,0.646854,0.459118,0.613846,0.191540,1.0,-0.002892,-12916.856807,418.730220,False
297,0.279626,0.306239,0.398689,0.274881,0.686909,0.541841,0.693218,0.107176,0.0,-0.003879,-13273.261269,139.627635,False
316,0.379418,0.515044,0.458128,0.029722,0.489059,0.663006,0.604753,0.476546,1.0,0.000291,-17706.144368,710.215223,False
408,0.292100,0.007941,0.396205,0.356095,0.983666,0.462480,0.727448,0.006768,0.0,-0.009836,-11099.897280,8.788080,False
490,0.225572,0.467351,0.215541,0.312105,0.531895,0.513960,0.767760,0.573396,1.0,-0.000661,-11357.570148,410.027062,False
511,0.000000,0.473154,0.638914,0.493375,0.523430,0.444087,0.652515,0.461494,1.0,-0.000545,-11955.127044,954.826737,False


In [12]:
librators

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My,a0,asd,qwe,librator
744,0.407484,0.481972,0.483035,0.226654,0.511986,0.599642,0.628015,0.250292,0.0,-0.000369,-15784.994863,393.035988,True
755,0.344075,0.482663,0.585246,0.093988,0.507013,0.682399,0.511253,0.042824,0.0,-0.000355,-20297.934799,81.239310,True
786,0.251559,0.478171,0.631134,0.464913,0.519643,0.466149,0.641634,0.328005,1.0,-0.000445,-12609.086162,670.454147,True
1731,0.435551,0.475182,0.305260,0.172861,0.521793,0.591739,0.705303,0.634831,1.0,-0.000505,-14034.819161,636.059284,True
2039,0.646570,0.481551,0.532923,0.028282,0.507021,0.678955,0.551434,0.265169,1.0,-0.000378,-19290.095992,458.677870,True
2587,0.540541,0.483068,0.518591,0.031113,0.505431,0.677547,0.562149,0.461494,1.0,-0.000347,-19010.941758,777.110535,True
2757,0.570686,0.482828,0.584187,0.032605,0.506285,0.697047,0.502989,0.112602,0.0,-0.000352,-20864.263751,213.230115,True
4152,0.572765,0.603151,0.419646,0.616986,0.396391,0.328955,0.828386,0.094107,1.0,0.002051,-7241.551785,128.559650,True
6656,0.624740,0.482968,0.569207,0.028069,0.506607,0.690553,0.516989,0.111785,0.0,-0.000349,-20372.162928,206.329929,True
6830,0.640333,0.487419,0.556497,0.122713,0.501045,0.671014,0.545337,0.562485,1.0,-0.000260,-19235.441595,1015.303910,True


In [275]:
non_librators.loc[0:]

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My,a0,asd,qwe
106,0.107370,0.306148,0.341880,0.120365,0.753764,0.573792,0.756499,0.022630,0.0,-0.003877,-17082.436935,208.641117
154,0.119199,0.985129,0.311172,0.697197,0.263698,0.285680,0.905694,0.002560,0.0,0.009701,-6404.095288,21.382908
176,0.173794,0.646078,0.456270,0.756285,0.515760,0.220136,0.881784,0.034330,0.0,0.002921,-5642.039587,419.414954
184,0.192903,0.864558,0.243859,0.068821,0.342499,0.625191,0.794082,0.001480,0.0,0.007290,-16927.880583,9.745459
199,0.203822,0.355432,0.475288,0.470614,0.727715,0.425043,0.765027,0.032830,1.0,-0.002892,-12916.856807,418.730220
297,0.244768,0.306073,0.280842,0.281816,0.757940,0.496953,0.810879,0.018370,0.0,-0.003879,-13273.261269,139.627635
316,0.332120,0.514596,0.322711,0.041249,0.608643,0.602279,0.759774,0.081681,1.0,0.000291,-17706.144368,710.215223
408,0.255687,0.008176,0.279092,0.361508,0.981872,0.427965,0.830652,0.001160,0.0,-0.009836,-11099.897280,8.788080
490,0.197452,0.466967,0.151830,0.318343,0.640967,0.472716,0.853940,0.098281,1.0,-0.000661,-11357.570148,410.027062
511,0.000000,0.472763,0.450059,0.496218,0.634579,0.411977,0.787365,0.079101,1.0,-0.000545,-11955.127044,954.826737


In [285]:
# librators = librators.assign(a2=((librators['a'] - RESONANCE_AXIS) ** 2) * 10 ** 5)
# librators.sort_values('%Name')
# non_librators = non_librators[
#     (non_librators['a'] < librators['a'].min()) |
#     (non_librators['a'] > librators['a'].max())
# ]
# non_librators

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My
106,7.48,3.170823,0.144660,0.062489,63.733326,162.988895,-104.807367,22.63,-2.0
154,7.61,3.184401,0.131878,0.344440,63.336366,90.525026,-70.743921,2.56,-2.0
176,8.21,3.177621,0.192273,0.373321,63.540540,74.039576,-76.203024,34.33,-2.0
184,8.42,3.181990,0.103860,0.037295,63.400196,175.916558,-96.226761,1.48,-2.0
199,8.54,3.171808,0.200189,0.233687,63.712226,125.576699,-102.860299,32.83,10.0
297,8.99,3.170821,0.119254,0.141405,63.736709,143.662833,-92.391755,18.37,-2.0
408,9.11,3.164864,0.118526,0.180358,63.918097,126.311625,-87.877084,1.16,-2.0
538,9.53,3.165175,0.133322,0.097777,63.906143,148.043153,-98.621396,1.47,-2.0
545,8.75,3.184426,0.180983,0.203700,63.328483,154.838383,-104.505179,1.46,-2.0
555,10.70,3.168278,0.185690,0.029383,63.813889,171.334658,-120.908651,39.04,10.0


In [204]:
temp = non_librators[(non_librators['a'] < librators['a'].max())]
temp[temp['a'] > librators['a'].min()]

,mag.,a,e,sin I,n,"g (""/yr)","s (""/yr)",LCEx1E6,My,a0,asd,qwe
316,0.332120,0.514596,0.322711,0.041249,0.608643,0.602279,0.759774,0.081681,1.0,0.000291,-17706.144368,710.215223
490,0.197452,0.466967,0.151830,0.318343,0.640967,0.472716,0.853940,0.098281,1.0,-0.000661,-11357.570148,410.027062
511,0.000000,0.472763,0.450059,0.496218,0.634579,0.411977,0.787365,0.079101,1.0,-0.000545,-11955.127044,954.826737
664,0.325751,0.507901,0.595561,0.311568,0.616713,0.624842,0.571402,0.150242,1.0,0.000157,-25858.758187,2392.054245
818,0.260237,0.474273,0.133744,0.508679,0.637551,0.362233,0.895535,0.102011,1.0,-0.000515,-8020.828297,376.701001
931,0.254777,0.503370,0.501007,0.352036,0.605701,0.543654,0.695595,0.128401,1.0,0.000067,-18448.990417,1722.578586
1072,0.401274,0.531008,0.506486,0.252415,0.587887,0.616580,0.653287,0.120081,1.0,0.000619,-22304.775079,1628.008951
1073,0.483167,0.497350,0.416863,0.042415,0.628460,0.638641,0.697089,0.089151,1.0,-0.000054,-21223.954369,997.674777
1099,0.346679,0.510381,0.622054,0.373485,0.587232,0.590920,0.573198,0.139001,1.0,0.000207,-24535.376874,2309.748658
1209,0.386715,0.481488,0.316939,0.194544,0.628357,0.559793,0.779063,0.090951,1.0,-0.000371,-15891.919047,777.113246


In [67]:
plt.figure(FOLDER, figsize=(15, 15))
x_col = 'a'
y_col = 'e'
# axes = plt.gca()
# axes.set_xlim([-0.003,0.003])
# axes.set_ylim([-0.5,5])
m_librators = cutoff_outliers(librators, ['a'])
plt.scatter(
    non_librators[x_col],
    non_librators[y_col],
    marker='^', color='r')
plt.scatter(
    m_librators[x_col], 
    m_librators[y_col],
    marker='o', color='b')
plt.xlabel(x_col, fontsize=18)
plt.ylabel(y_col, fontsize=18)
plt.legend(['non librators', 'librators'], loc='upper left')
plt.title(FOLDER, fontsize=20)
# plt.show()
m_librators.shape

(40, 13)

In [39]:
qwe = catalog_data[['a', 'n']]
qwe = qwe.assign(qwe=1)
qwe
# asd = catalog_data[['librator']].values
# qwe
# asd[np.where(asd==True)]

,a,n,qwe
106,3.170823,63.733326,1
154,3.184401,63.336366,1
176,3.177621,63.540540,1
184,3.181990,63.400196,1
199,3.171808,63.712226,1
297,3.170821,63.736709,1
316,3.174991,63.615776,1
408,3.164864,63.918097,1
490,3.174039,63.641959,1
511,3.174155,63.636785,1


In [ ]:
clf = xgb.sklearn.XGBClassifier(n_estimators=10, max_depth=2, nthread=4,
                                gamma=4, learning_rate=.1, reg_alpha=0, reg_lambda=1)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=35, p=1, weights='distance', n_jobs=4)
targets = catalog_data['librator']
# targets
features = dataset[[
    'a', 
    'sin I',
    'e',
#     'My',
#     'g ("/yr)',
#     's ("/yr)',
    'n',
#     'LCEx1E6',
#     'qwe',
#     'asd'
]]
def oversampling(x_train, y_train):
    true_class_mask = np.where(y_train==True)
    true_class_objects = x_train[true_class_mask]
    true_class_targets = y_train[true_class_mask]
    for _ in range(1000):
        x_train = np.vstack((x_train, true_class_objects))
        y_train = np.hstack((y_train, true_class_targets))
    return x_train, y_train

def smote(x_train, y_train):
    from imblearn.over_sampling import SMOTE
    sm = SMOTE(ratio=0.99, random_state=42, k=31, n_jobs=4)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    return x_train, y_train
# features.values
kf = skl.cross_validation.KFold(targets.shape[0], 5, shuffle=True, random_state=42)
precision, recall, accuracy, TP, FP, TN, FN = classify(clf, kf, features.values, targets.values, oversampling)
pd.DataFrame([[precision, recall, accuracy, TP, FP, TN, FN]],
             columns=['precision', 'recall', 'accuracy', 'TP', 'FP', 'TN', 'FN'])

In [127]:
qwe = [1,2,3]
np.transpose(np.array([qwe]))

array([[1],
       [2],
       [3]])